In [1]:
import numpy as np
import keras
import tensorflow as tf
import tensorflow_datasets as tfds

2023-10-11 13:03:51.937128: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# mnist_train= tfds.load(name='fashion_mnist', split='train')
mnist_test= tfds.load(name='fashion_mnist', split='test')

In [3]:
print(type(mnist_test))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [4]:
print(len(mnist_test))

10000


In [5]:
data, info = tfds.load('fashion_mnist', with_info=True)
print(info)

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_dir='/Users/jasonmiller/tensorflow_datasets/fashion_mnist/3.0.1',
    file_format=tfrecord,
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
    

In [6]:
(train_images,train_labels),(test_images,test_labels) = tfds.load(
    'fashion_mnist', split=['train','test'], batch_size=-1, as_supervised=True)

In [7]:
print(len(train_images))
print(type(train_images))
print(type(train_images[0]))

60000
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [8]:
image1 = train_images[0]
print('one row of grayscale pixel values')
print(image1[3])

normalizer = tf.cast(255,tf.float32)
norm_images = tfds.as_numpy(train_images)/normalizer

image1 = norm_images[0]
print('same row of normalized pixel values')
print(image1[3])

one row of grayscale pixel values
tf.Tensor(
[[  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [ 26]
 [ 92]
 [ 69]
 [ 68]
 [ 75]
 [ 75]
 [ 71]
 [ 74]
 [ 83]
 [ 75]
 [ 77]
 [ 78]
 [ 74]
 [ 74]
 [ 83]
 [ 77]
 [108]
 [ 34]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]], shape=(28, 1), dtype=uint8)
same row of normalized pixel values
tf.Tensor(
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.10196079]
 [0.36078432]
 [0.27058825]
 [0.26666668]
 [0.29411766]
 [0.29411766]
 [0.2784314 ]
 [0.2901961 ]
 [0.3254902 ]
 [0.29411766]
 [0.3019608 ]
 [0.30588236]
 [0.2901961 ]
 [0.2901961 ]
 [0.3254902 ]
 [0.3019608 ]
 [0.42352942]
 [0.13333334]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]], shape=(28, 1), dtype=float32)


In [9]:
def augment(image,label):
    image = tf.cast(image, tf.float32)
    image = image/255
    image = tf.image.random_flip_left_right(image)
    return image,label

In [16]:
train_set = tfds.load(
    'fashion_mnist', split='train', as_supervised=True)
#example = train_set[0]
print(type(train_set))
stop = 10
for image, label in train_set:
    if stop == 0:
        break
    stop -= 1
    nimage = np.asarray(image)
    value = int(label)
    print(np.shape(nimage), 'label', value)

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>
(28, 28, 1) label 2
(28, 28, 1) label 1
(28, 28, 1) label 8
(28, 28, 1) label 4
(28, 28, 1) label 1
(28, 28, 1) label 9
(28, 28, 1) label 2
(28, 28, 1) label 2
(28, 28, 1) label 0
(28, 28, 1) label 2


2023-10-11 13:07:36.618089: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [17]:
augmented_set = train_set.map(augment)

In [19]:
print(type(augmented_set))
stop = 10
for image, label in augmented_set:
    if stop == 0:
        break
    stop -= 1
    nimage = np.asarray(image)
    value = int(label)
    print(np.shape(nimage), 'label', value)

<class 'tensorflow.python.data.ops.map_op._MapDataset'>
(28, 28, 1) label 2
(28, 28, 1) label 1
(28, 28, 1) label 8
(28, 28, 1) label 4
(28, 28, 1) label 1
(28, 28, 1) label 9
(28, 28, 1) label 2
(28, 28, 1) label 2
(28, 28, 1) label 0
(28, 28, 1) label 2


2023-10-11 13:08:45.864104: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
